In [1]:
import pandas as pd
import numpy as np
import datetime

# Retrieve River Flows
flow = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentWater.csv')
# Retrieve Precip
rain = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/CurrentPrecip.csv',index_col=0)
# Retrieve Forecast
rainforecast = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Ingestion/NewWeatherForecast.csv',index_col=0)
# Retrieve River References
reference = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')

rainforecast['EndDate'] = pd.to_datetime(rainforecast.endTime).dt.date
rainforecast['StartDate'] = pd.to_datetime(rainforecast.startTime).dt.date

# Join on Date
flow = flow.set_index('Name').join(reference[['USGS Name','Name']].set_index('USGS Name')).reset_index()
flow['CFS'] = flow.Value
flow['DateTime'] = pd.to_datetime(flow.DateTime)
flow['Date'] = flow.DateTime.dt.date
flow = flow[['Date','Name','CFS']].groupby(['Date','Name']).mean().reset_index()
flow['Date_Name'] = flow.Date.astype(str)+'_'+flow.Name


rain = rain.drop_duplicates()
rain['Date'] = pd.to_datetime(rain.Date).dt.date
rain['Date_Name'] = rain.Date.astype('str')+'_'+rain.Name
rain = rain[['Date_Name','Date','Name','Precip']].groupby(['Date','Name','Date_Name']).mean().reset_index()

df = flow.set_index('Date_Name').join(rain[['Date_Name','Precip']].set_index('Date_Name'))
RiverList = df.Name.unique()
full = df.loc[df.Precip.isnull()==False]

# Lag Precip and Forecast data by correlation strength to river flow
RiverData = pd.DataFrame()
for n in RiverList:
    s = 0
    CorrList = list()
    while s < 10:
        d = full[['CFS','Precip']].loc[full.Name==n]
        d['Precip'] = d['Precip'].shift(s)
        r = d.dropna()
        corr = r.Precip.corr(r.CFS)
        CorrList.append([s,corr])
        s+=1
    Corrs = pd.DataFrame(CorrList,columns=['PrecipLag','Corr'])
    BestShift = Corrs.sort_values(by='Corr',ascending=False).head(1)
    BestShift['Name'] = n
    RiverData = RiverData.append(BestShift)

    
# Format data like example below
headers = ['Date','RiverName','DayQuality','DayAvgCFS','Precip','NextDayForecast','Forecast_2','Forecast_3','Forecast_4','Forecast_5','Forecast_6','Forecast_7','Forecast_8','Forecast_9','Forecast_10']
d = [['11-01-2022',0,0.88,500,2,0,0,2,3,4,5,0,2,0,3]]
model = pd.DataFrame(data=d,columns=headers)



# Bring in river level data
rivers = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')[['Name','RiverName']]

# Create correlation coefficients for the following attributes
# corr(CFS & Precip)
# corr(CFS & Forecast)
# corr(Precip & Forecast)
# Append those coefficients to river level dataset
rivers['River__Rain_Coeff'] = None
rivers['River__Forecast_Coeff'] = None
rivers['Rain__Forecast_Coeff'] = None
rivers.head()

c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Scott\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Name,RiverName,River__Rain_Coeff,River__Forecast_Coeff,Rain__Forecast_Coeff
0,Russian River HaciendaBridge,Russian,None,None,None
1,Navarro River,Navarro,None,None,None
2,Noyo River,Noyo,None,None,None
3,San Lorenzo River Upper,San Lorenzo,None,None,None
4,Mad River,Mad,None,None,None


Next Steps:
- Ensure datasets join properly
- Answer the question:
    - How to relate the daily rain forecast data (Date,River,Forcast) to the core historical dataset (Date,River,Histical Metrics)? Specifically, how should the joined data be structured for modeling?
    - Create a single forecast for each day based on the avg of the forecasts available.
    

- Need R-value of >.75 when evaluating CFS by Precipition 
- Based on the historical relationship between CFS and precipitation, how confidently can I predict CFS based on the forecasted precipitation?